# Customizing Model 
* 이 노트북은 딥러닝 모형을 커스텀 하기 위한 학습을 진행하기 위해 생성되었습니다. 
* 특히, 해당 노트북은 커스텀하게 구조화된 모형을 학습 시키기 위한 방법을 고안하기 위해 만들어졌으며
* 주로 keras.fit() 메소드를 low Level에서 개인화 하기 위한 학습을 진행합니다. 

In [1]:
import tensorflow as tf 
from tensorflow import keras 

## customize Keras.fit( ) 공식문서 공부하기
* 해당 파트는 [Keras 공식문서 || Customizing what happens in fit()](https://keras.io/guides/customizing_what_happens_in_fit/)의 내용을 기반으로 작성하였습니다. 


* 커스텀 모형을 학습시키는 방식에는 크게 tf.GradientTape메소드와 반복문의 조합을 통한 방식과  
* keras의 fit 메소드를 사용하는 두가지 방식이 존재합니다. 
  
* GradientTape을 사용한다면 모형의 세부사항까지 통제가 가능하다는 장점을 갖지만, callback과 built-in distribution support, step fusing과 같은 편리한 기능의 이점을 활용하려면 fit 메소드를 응용할 수 있는 능력이 요구됩니다. 

* fit( ) 메소드를 커스텀 하기 위해서는 Model class의 training step function을 Override하면 된다고 합니다. 
* 해당 부분은 fit( ) 메소드에 의해 매번 배치시 마다 호출되는 함수입니다.  
    (아래의 예시에서는 subclassing예제를 보여주지만, functional api, sequential Model subclassed model 어디에서든 사용가능합니다.)

### first example
* keras.Model을 subclassing하는 새로운 모형을 생성합니다. 
* 이때, 단순히 **train_step(self,data)** 부분을 오버라이딩 합니다.


In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # 데이터를 unpack합니다. 해당 부분의 형태는 
        # fit()을 통해 모형의 입력으로 사용된 데이터의 형태에 따라 달라집니다. 
        # fit(x,y,...)형태의 입력을 가정합니다. 
        x,y = data 
        
        # loss를 계산합니다. 
        with tf.GradientTape() as tape:
            y_pred = self(x, training = True) # 순전파를 통한 예측 
            # loss값을 계산합니다. 
            # loss함수는 compile()을 통해 입력된 loss함수를 기준으로 합니다. 
            loss = self.compiled_loss(y,y_pred, regularization_losses = self.losses)
        
        # 기울기를 계산합니다. 
        # 대상 노드 정보를 담습니다. 
        trainable_vars = self.trainable_variables 
        # 각각의 기울기를 구해줍니다. 
        gradients = tape.gradient(loss, trainable_vars)
        
        # 구한 기울기를 경사하강법을 적용해 각각의 간선에 반영해줍니다. 
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # metrics를 업데이트합니다. (이때, loss를 추적하는 metrics도 같이 업데이트해줍니다.)
        self.compiled_metrics.update_state(y,y_pred)
        
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

##### 상세설명
1. **data** : fit()메소드에 입력되는 데이터를 의미합니다. 
    - 만약 fit(x,y, ...)을 입력으로 사용했다면 data는 (x,y)의 튜플이 될 것입니다. 
    - 만약 tf.keras.Dataset으로 생성한 dataset을 입력으로 fit(dataset, ...)을 사용했다면 data에는 각각의 배치에서 dataset에 의해 산출되는 형태가 될 것입니다. 

In [ ]:
# 생성한 모형을 사용해봅니다. 
import numpy as np 

# 위에서 subclassing을 진행한 customModel의 인스턴스를 생성합니다. 
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
# 이 부분에서 모형을 사용합니다. 
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# fit은 평소 사용하던데로 진행합니다. 
x = np.random.random((1000,32))
y = np.random.random((1000,1))
model.fit(x,y, epochs=3)


### Going lower-level 
* 기본적으로는 compile()메소드에 어떠한 인자를 입력하지 않고, 기본적으로 모든 인자를 train_step에 있는 기본 값으로 설정할 수도 있습니다. 
* 아래의 예시는 단순히 compile()메소드를 옵티마이져를 구성하기 위해서만 사용하는 경우를 상정한 모델링 방식을 예시로 합니다. 

1. loss를 추적하고 MAE 스코어를 추적하기 위한 Metric 인스턴스를 생성합니다. 
2. train_step과정에서 .update_state() 메소드를 사용해 해당 값들을 갱신해주고 .result()메소드를 통해 학습 과정중에서 해당 값들의 최신 값을 조회하고 이를 진행상태 bar에 표시해줌과 동시에 callback에 사용될 수 있도록 해줍니다. 
3. 주의사항은 각각의 에포크 사이에 필수적으로 reset_state를 통해 값을 초기화 시켜주어야 한다는 점입니다. (그렇지 않으면 result()가 뱉어내는 값은 처음 학습을 시작할때부터 지금까지의 평균치를 뱉어낼 것입니다.) 
*  Thankfully, the framework can do that for us: just list any metric you want to reset in the **metrics property** of the model. The model will call reset_states() on any object listed here at the beginning of each fit() epoch or at the beginning of a call to evaluate().

In [ ]:
# 사용하고자하는 각각의 metrics를 class밖의 전역변수로 선언하고, class에 넘겨줍니다. 
# 이전에는 self.metrics를 통해 compile시 들어온 metric을 사용했지만, 이번에는 직접 입력하고 컴파일 옵션에는 별도로 지정하지 않습니다. 
loss_tracker = keras.metrics.Mean(name='loss')
mae_metric = keras.metrics.MeanAbsoluteError(name='mae')

# 서브클레싱을 진행합니다. 이때, metric을 전역변수에 선언된 내용을 기반으로 진행합니다. 
class CustomModel(keras.Model):
    
    # 해당 부분은 fit 메소드가 한번의 에포크마다 사용하는 부분입니다. 
    def train_step(self,data):
        # 입력된 데이터의 형태에 따라 unpack을 진행합니다. 
        x,y = data 
        
        # loss를 계산합니다.(1번의 에포크 안에서 여러번 계산(한번의 전파시 한번의 갱신)) 
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) # 순전파 
            # loss를 계산합니다. 
            loss = keras.losses.mean_squared_error(y,y_pred)
        
        # 기울기를 계산합니다. 
        trainable_vars = self.trainable_variables 
        gradients = tape.gradient(loss, trainable_vars)
        
        # 가중치를 업데이트합니다. 
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # 직접 생성한 메트릭스를 계산합니다. 
        # 한번의 에포크 안에서 발생한 여러번의 연산 결과를 누적 평균 및 누적 mae 를 진행 
        # 한번의 에포크가 끝나면 return 후 reset_state()
        loss_tracker.update_state(loss)
        mae_metric.update_state(y,y_pred)
        
        return {"loss":loss_tracker.result(), "mae":mae_metric.result()}
    
    @property 
    def metrics(self):
        # 해당부분에 우리가 정의한 'metric'들의 list를 두면 
        # 모델이 자동적으로 각각의 에포크 사이에 reset_state()를 진행해줍니다. 
        return [loss_tracker, mae_metric]

        
        

In [ ]:
# custom model의 인스턴스를 생성합니다. 
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# 모형에 메트릭스를 정의하지 않습니다. 
model.compile(optimizer="adam")

x= np.random.random((100000,32))
y = np.random.random((100000,1))
model.fit(x,y, epochs=5)

### Supporting sample_weight & class_weight
You may have noticed that our first basic example didn't make any mention of sample weighting. If you want to support the fit() arguments sample_weight and class_weight, you'd simply do the following:
  
  
* Unpack sample_weight from the data argument
* Pass it to compiled_loss & compiled_metrics (of course, you could also just apply it manually if you don't rely on compile() for losses & metrics)
* That's it. That's the list.

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        if len(data) == 3:
            x, y, sample_weight = data
        else:
            sample_weight = None
            x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.compiled_loss(
                y,
                y_pred,
                sample_weight=sample_weight,
                regularization_losses=self.losses,
            )

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        self.compiled_metrics.update_state(y, y_pred, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# You can now use sample_weight argument
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=3)

* 해당 부분은 추후 복습을 진행할 예정입니다. 

### Providing your own evaluation step
What if you want to do the same for calls to model.evaluate()? Then you would override test_step in exactly the same way. Here's what it looks like:



In [ ]:
class CustomModel(keras.Model):
    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(loss="mse", metrics=["mae"])

# Evaluate with our custom test_step
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.evaluate(x, y)

* 해당 부분 및 GAN예제 또한 추후 보강 예정입니다. 

## Make Customized Model
* 학습을 위한 테스트 모형을 구성합니다. 
* 모형의 구조는 2SLS 계량 모형을 구조화 하여 신경망으로 만든 테스트 모델입니다. 
[main](https://frhyme.github.io/machine-learning/a_model_in_keras/)  [sub1](https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/)

### 방법1 

In [9]:
def model():
    
    # first stage 
    input1 = tf.keras.layers.Input(shape= (5,))
    x = tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu)(input1)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(32, activation=tf.nn.leaky_relu)(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(1)(x)
    x = keras.Model(inputs= input1, outputs= x)
    
    # second stage  
    input2 = tf.keras.Input(shape= (4,))
    y = tf.keras.layers.Dense(4)(input2)
    y = keras.Model(inputs= input2, outputs=y) 
    
    concat = tf.keras.layers.concatenate([x.output, y.output])
    
    z = tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu)(concat)
    z = tf.keras.layers.Dropout(0.4)(z)
    z = tf.keras.layers.Dense(32, activation=tf.nn.leaky_relu)(z)
    z = tf.keras.layers.Dropout(0.4)(z)
    z = tf.keras.layers.Dense(1)(z)
    
    model = keras.Model(inputs= [x.input, y.input], outputs=z)
    
    return model
    
model = model()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 64)           384         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 64)           0           dense_5[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 32)           2080        dropout_4[0][0]                  
____________________________________________________________________________________________

* 해당 방식은 전체 학습과정을 완료한 이후, 최종적인 ouput과 실제 output만을 비교하여 학습한다는 단점을 지닙니다. 
* 2SLS모형 처럼 first stage에서도 실제값과 예측값의 차이를 활용한 가중치 업데이트가 가능하도록 모형을 설계해야 합니다. 

* 또한, 모형은 생성하였으나 이를 학습시키기 위한 방안에 대해 고민해보아야 합니다.

### 방법2 - Keras의 Multi-input Multi-output 모형 구조 응용하기
* 해당 방식은 keras의 공식 문서를 인용한 블로그의 내용을 참조하여 구성하였습니다. [keras의 model을 파봅시다](https://frhyme.github.io/machine-learning/a_model_in_keras/)[Keras Multi(input, output)모델 생성 방법](https://deeptak.tistory.com/7)
* 여러개의 input과 output을 갖는 모형을 구조화 합니다. 
* 해당 방식은 당초 원하던, 도구 변수를 통한 예측값의 도출과. 실제값과의 비교를 통한 가중치 수정 방식을 갖는다는 점에서 의의를 갖습니다. 

In [2]:
# first stage input layer 
first_input = keras.layers.Input(shape=(4,), name='first_input')
stage1 = keras.layers.Dense(64, activation=tf.nn.leaky_relu)(first_input)
stage1 = keras.layers.Dropout(0.4)(stage1)
stage1 = keras.layers.Dense(32, activation=tf.nn.leaky_relu)(stage1)
stage1 = keras.layers.Dropout(0.4)(stage1)

# stage1의 중간 출력 노드 
auxiliary_output = keras.layers.Dense(1, activation='linear', name='aux_output')(stage1)

# stage1에서 다음 스테이지로 넘어갈 output 
stage1_out = keras.layers.Dense(1)(stage1)
    
# second stage input layer 
second_input = keras.layers.Input(shape=(5,), name='second_input')

# concat 이후 main stream으로 
x = keras.layers.concatenate([second_input, stage1_out])

x = tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu)(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(32, activation=tf.nn.leaky_relu)(x)
x = tf.keras.layers.Dropout(0.4)(x)
main_output = tf.keras.layers.Dense(1, activation='linear', name='main_output')(x)

# 전체 모형 생성 
model = keras.Model(inputs=[first_input, second_input], outputs=[main_output,auxiliary_output])

# 모델 컴파일 
model.compile(optimizer='adam', loss='mae')

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_input (InputLayer)        [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           320         first_input[0][0]                
__________________________________________________________________________________________________
dropout (Dropout)               (None, 64)           0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           2080        dropout[0][0]                    
______________________________________________________________________________________________

In [7]:
import numpy as np 

first = np.random.random((1000,4))
second = np.random.random((1000,5))

z = np.random.random((1000,1))
y = np.random.random((1000,1))

In [8]:
model.fit({'first_input': first, 'second_input': second}, 
          {'main_output': y, 'aux_output': z}, 
          epochs=50, batch_size=32)

Epoch 1/50
32/32 [==============================] - 1s 2ms/step - loss: 0.8123 - main_output_loss: 0.3655 - aux_output_loss: 0.4468
Epoch 2/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5995 - main_output_loss: 0.2978 - aux_output_loss: 0.3016
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5602 - main_output_loss: 0.2836 - aux_output_loss: 0.2766
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5526 - main_output_loss: 0.2802 - aux_output_loss: 0.2725
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5540 - main_output_loss: 0.2721 - aux_output_loss: 0.2819
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5345 - main_output_loss: 0.2667 - aux_output_loss: 0.2678
Epoch 7/50
32/32 [==============================] - 0s 2ms/step - loss: 0.5352 - main_output_loss: 0.2660 - aux_output_loss: 0.2693
Epoch 8/50
32/32 [==============================] - 0s 2ms/step - loss: 0.52

* 하지만, 학습과정에서 일어나는 현상에 대해서는 아직 공부가 더 필요합니다. 
* 만약 학습 방식이 모든 stage를 마치고 가중치를 업데이트하는 방식이라면, 당초 예상하던 학습 형태와는 거리가 있다고 생각합니다. 

## 앞단의 모형을 먼저 학습시키고, 연결하는 방식은 어떨까? 

[stackoverflow | How to concatnate two pretrained models](https://stackoverflow.com/questions/66852496/how-to-concatenate-two-pre-trained-models-in-keras)

## 한번의 트레이닝에서 모형 전체를 학습시킬 순 없을까?